In [2]:
import numpy as np
from math import pow, exp
from numpy.linalg import matrix_rank

def linearGen(x0, a, c, m):
    xn = x0
    while True:
        yield xn
        xn = (a * xn + c) % m

def mGen(xg, yg, m, k):
    v = [next(xg) for _ in range(k)]
    while True:
        x, y = next(xg), next(yg)
        j = int(y * k / m)
        yield v[j]
        v[j] = x

def discreteVal(g, m):
    while True:
        yield next(g) % m

def lfsrGen(S, c):
    while True:
        yield S[0]
        S = np.append(S, np.mod(np.sum(S * c), 2))[1:]

def ssGen(g):
    a, b = next(lfsr), next(lfsr)
    while True:
        if a == 1:
            yield b
        a, b = b, next(lfsr)

def binaryMatrixRank(seq, M, Q):
    fm = 0
    fmm1 = 0
    N = len(seq) // (M*Q)
    for block in range(N):
        mat = np.array( seq[block * M * Q: (block + 1) * M * Q] ).reshape( (M, Q) )
        r = matrix_rank( mat )
        if r == M:
            fm += 1
        elif r == M - 1:
            fmm1 += 1
    print(fm,fmm1,N - fm - fmm1)
    cf = [(fm, 0.2888), (fmm1, 0.5776), (N - fm - fmm1, 0.1336)]
    xsq = 0.0
    for x, y in cf:
        xsq += (x - y*N)**2 / (y*N)
    return exp(-xsq/2)

g1 = linearGen(2, 34, 101, 99)
g2 = linearGen(32423, 945, 25725, 2**16)
g3 = mGen(g1, g2, 2**16, 64)
g4 = mGen(g1, g2, 2**16, 256)
gs = [g1,g2,g3,g4]
dgs = []
for g in gs:
    dgs.append(discreteVal(g, 2))
S = np.random.randint(2, size=64)
c = np.array([1 if x in [0, 59, 60, 62, 63] else 0 for x in range(64)])
lfsr = lfsrGen(S, c)
ss = ssGen(lfsr)
ggs = [lfsr, ss] + dgs

for g in ggs:
    seq = [next(g) for _ in range(1024)]
    print(binaryMatrixRank(seq, 3, 3))

42 58 13
0.15042943614575494
31 74 8
0.10096505599019473
10 20 83
2.8341457344947158e-77
0 113 0
1.1365213127520946e-18
40 60 13
0.3043784693116464
33 66 14
0.9550557467169228
